In [1]:
%pip install nltk
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import sagemaker
import pandas as pd
import matplotlib.pyplot as plt
import sagemaker, boto3, json
from sagemaker import get_execution_role
import nltk
nltk.download('punkt')
%matplotlib inline

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[nltk_data] Downloading package punkt to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
region = boto3.Session().region_name

# Specify S3 bucket and prefix where you have uploaded email_dataset.csv
#training_data_bucket = "spam1"
bucket = 'spam1'
data_key = 'email_dataset.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
print(data_location)

#training_dataset_s3_path = f"s3://{training_data_bucket}/email_dataset.csv"
#print(training_dataset_s3_path)

output_bucket = sess.default_bucket()
output_prefix = ""

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

s3://spam1/email_dataset.csv


In [4]:
# load data
df = pd.read_csv(data_location)
df.head()

,Category,Message
0,HAM,just wanted to check with you if you'll be sha...
1,HAM,"Hi,Job Title:- SAP ABAP Consultant Experience-..."
2,SPAM,Winter is here and so are the grand holidays
3,HAM,"Hi,We are looking for an expert in SAP for our..."
4,HAM,Your prepaid recharge is now successful.


In [5]:
# Check if dataset is balanced or not
df['Category'].value_counts()

Category
HAM     42
SPAM    38
Name: count, dtype: int64

In [6]:
def tokenize(message):
    # delete quotation marks and commas , apply tokenization and join back into a string separating by spaces
    return ' '.join([str(token) for token in nltk.word_tokenize(str(message).replace(',', '').replace('"', '').lower())])
    
def prepare_data(df):
    df['Category'] = df['Category'].map(lambda category : '__label__{}'.format(str(category).replace('__label__', '')))
    df['Message'] = df['Message'].map(lambda message : tokenize(message)) 
    return df

df_final = df[['Category', 'Message']].reset_index(drop=True)
df_final = prepare_data(df_final)
df_final.head()

,Category,Message
0,__label__HAM,just wanted to check with you if you 'll be sh...
1,__label__HAM,hijob title : - sap abap consultant experience...
2,__label__SPAM,winter is here and so are the grand holidays
3,__label__HAM,hiwe are looking for an expert in sap for our ...
4,__label__HAM,your prepaid recharge is now successful .


In [7]:
# Split data into train and validation
from sklearn.model_selection import train_test_split
df_train, df_validation = train_test_split(df_final, 
                                           test_size=0.10,
                                           stratify=df_final['Category'])

In [8]:
#upload transformed data to S3 bucket
train_path = './train.csv'
df_train[['Category', 'Message']].to_csv(train_path, index=False, header=False, sep=' ')

validation_path = './validation.csv'
df_validation[['Category', 'Message']].to_csv(validation_path, index=False, header=False, sep=' ')

#Specify S3 bucket prefix
train_s3_uri = sess.upload_data(bucket=bucket, key_prefix='trainig', path=train_path)
validation_s3_uri = sess.upload_data(bucket=bucket, key_prefix='validation', path= validation_path)

In [9]:
image_uri = sagemaker.image_uris.retrieve(
    region=region,
    framework='blazingtext'
)

In [10]:
estimator = sagemaker.estimator.Estimator(image_uri=image_uri, 
    role=aws_role, 
    instance_count=1, 
    instance_type='ml.m5.large',
    volume_size=30,
    max_run=7200,
    disable_profiler=True,                                      
    sagemaker_session=sess
)

In [11]:
#Hyperparameter
estimator.set_hyperparameters(mode='supervised',   
                              epochs=10,          
                              learning_rate=0.01,  
                              min_count=2,                          
                              vector_dim=300,      
                              word_ngrams=3)  

In [12]:
train_data = sagemaker.inputs.TrainingInput(
    train_s3_uri, 
    distribution='FullyReplicated', 
    content_type='text/plain', 
    s3_data_type='S3Prefix'
)
validation_data = sagemaker.inputs.TrainingInput(
    validation_s3_uri, 
    distribution='FullyReplicated', 
    content_type='text/plain', 
    s3_data_type='S3Prefix'
)

data_channels = {
    'train': train_data,
    'validation': validation_data 
}

In [13]:
estimator.fit(
    inputs=data_channels,
    wait=True
)

INFO:sagemaker:Creating training-job with name: blazingtext-2024-05-10-15-02-59-209


2024-05-10 15:02:59 Starting - Starting the training job...
2024-05-10 15:03:15 Starting - Preparing the instances for training...
2024-05-10 15:03:43 Downloading - Downloading input data...
2024-05-10 15:04:23 Downloading - Downloading the training image...
2024-05-10 15:04:39 Training - Training image download completed. Training in progress.Arguments: train
[05/10/2024 15:04:43 WARNING 139733088479040] Loggers have already been setup.
[05/10/2024 15:04:43 WARNING 139733088479040] Loggers have already been setup.
/opt/amazon/python3.8/lib/python3.8/subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
[05/10/2024 15:04:43 INFO 139733088479040] nvidia-smi took: 0.025179386138916016 secs to identify 0 gpus
[05/10/2024 15:04:43 INFO 139733088479040] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is 

In [14]:
#Get the accuracy of the train and validation dataset
estimator.training_job_analytics.dataframe()

,timestamp,metric_name,value
0,0.0,train:accuracy,0.9444
1,0.0,validation:accuracy,0.6250


In [ ]:
text_classifier = estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m5.large',
                                   serializer=sagemaker.serializers.JSONSerializer(),
                                   deserializer=sagemaker.deserializers.JSONDeserializer())
print()
print('Endpoint name:  {}'.format(text_classifier.endpoint_name))

INFO:sagemaker:Creating model with name: blazingtext-2024-05-10-15-07-15-934
INFO:sagemaker:Creating endpoint-config with name blazingtext-2024-05-10-15-07-15-934
INFO:sagemaker:Creating endpoint with name blazingtext-2024-05-10-15-07-15-934


--

In [ ]:
messages = [
                # Spam
                'Click on below link, provide your details and win this award' ,
                'Best summer deal here',
                #ham
                'See you in the office on Friday.'

]

tokenized_message = [' '.join(nltk.word_tokenize(mesaage)) for mesaage in messages]
payload = {"instances" : tokenized_message}
print(payload)

In [ ]:
predictions = text_classifier.predict(data=payload)
for prediction in predictions:
    predicted_class = prediction['label'][0].lstrip('__label__')
    print('SPAM' if predicted_class == '1' else 'HAM')

In [ ]:
#text_classifier.delete_endpoint()